In [1]:
import sys
sys.path.insert(0, '../ELINA/python_interface/')

import numpy as np
import re
import csv
from elina_box import *
from elina_interval import *
from elina_abstract0 import *
from elina_manager import *
from elina_dimension import *
from elina_scalar import *
from elina_interval import *
from elina_linexpr0 import *
from elina_lincons0 import *
import ctypes
from ctypes.util import find_library
from gurobipy import *
import time

In [2]:
# Import for debugging in jupyter notebook
from IPython.core.debugger import set_trace #TODO remove at end.

In [3]:
libc = CDLL(find_library('c'))
cstdout = c_void_p.in_dll(libc, 'stdout')

In [4]:
class layers:
    def __init__(self):
        self.layertypes = []
        self.weights = []
        self.biases = []
        self.numlayer = 0
        self.ffn_counter = 0
        self.rank = []
        self.use_LP = []

In [5]:
def parse_bias(text):
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    v = np.array([*map(lambda x: np.double(x.strip()), text[1:-1].split(','))])
    #return v.reshape((v.size,1))
    return v

In [6]:
def parse_vector(text):
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    v = np.array([*map(lambda x: np.double(x.strip()), text[1:-1].split(','))])
    return v.reshape((v.size,1))
    #return v

In [7]:
def balanced_split(text):
    i = 0
    bal = 0
    start = 0
    result = []
    while i < len(text):
        if text[i] == '[':
            bal += 1
        elif text[i] == ']':
            bal -= 1
        elif text[i] == ',' and bal == 0:
            result.append(text[start:i])
            start = i+1
        i += 1
    if start < i:
        result.append(text[start:i])
    return result

In [8]:
def parse_matrix(text):
    i = 0
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    return np.array([*map(lambda x: parse_vector(x.strip()).flatten(), balanced_split(text[1:-1]))])

In [9]:
def parse_net(text):
    lines = [*filter(lambda x: len(x) != 0, text.split('\n'))]
    i = 0
    res = layers()
    while i < len(lines):
        if lines[i] in ['ReLU', 'Affine']:
            W = parse_matrix(lines[i+1])
            b = parse_bias(lines[i+2])
            res.layertypes.append(lines[i])
            res.weights.append(W)
            res.biases.append(b)
            res.numlayer+= 1
            res.rank.append(np.zeros((W.shape[0],1)))
            res.use_LP.append(np.full((W.shape[0],1), False))
            i += 3
        else:
            raise Exception('parse error: '+lines[i])
    return res

In [10]:
def parse_spec(text):
    text = text.replace("[", "")
    text = text.replace("]", "")
    with open('dummy', 'w') as my_file:
        my_file.write(text)
    data = np.genfromtxt('dummy', delimiter=',',dtype=np.double)
    low = np.copy(data[:,0])
    high = np.copy(data[:,1])
    return low,high

In [11]:
def get_perturbed_image(x, epsilon):
    image = x[1:len(x)]
    num_pixels = len(image)
    LB_N0 = image - epsilon
    UB_N0 = image + epsilon
     
    for i in range(num_pixels):
        if(LB_N0[i] < 0):
            LB_N0[i] = 0
        if(UB_N0[i] > 1):
            UB_N0[i] = 1
    return LB_N0, UB_N0

In [12]:
def generate_linexpr0(weights, bias, size):
    linexpr0 = elina_linexpr0_alloc(ElinaLinexprDiscr.ELINA_LINEXPR_DENSE, size)
    cst = pointer(linexpr0.contents.cst)
    elina_scalar_set_double(cst.contents.val.scalar, bias)
    for i in range(size):
        elina_linexpr0_set_coeff_scalar_double(linexpr0,i,weights[i])
    return linexpr0

In [13]:
def analyze(nn, LB_N0, UB_N0, label):   
    num_pixels = len(LB_N0)
    nn.ffn_counter = 0
    numlayer = nn.numlayer 
    man = elina_box_manager_alloc()
    itv = elina_interval_array_alloc(num_pixels)
    for i in range(num_pixels):
        elina_interval_set_double(itv[i],LB_N0[i],UB_N0[i])

    ## construct input abstraction
    element = elina_abstract0_of_box(man, 0, num_pixels, itv)
    elina_interval_array_free(itv,num_pixels)
    for layerno in range(numlayer):
        if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
            weights = nn.weights[nn.ffn_counter]
            biases = nn.biases[nn.ffn_counter]
            dims = elina_abstract0_dimension(man,element)
            num_in_pixels = dims.intdim + dims.realdim
            num_out_pixels = len(weights)

            dimadd = elina_dimchange_alloc(0,num_out_pixels)    
            for i in range(num_out_pixels):
                dimadd.contents.dim[i] = num_in_pixels
            elina_abstract0_add_dimensions(man, True, element, dimadd, False)
            elina_dimchange_free(dimadd)
            np.ascontiguousarray(weights, dtype=np.double)
            np.ascontiguousarray(biases, dtype=np.double)
            var = num_in_pixels
            # handle affine layer
            for i in range(num_out_pixels):
                tdim= ElinaDim(var)
                linexpr0 = generate_linexpr0(weights[i],biases[i],num_in_pixels)
                element = elina_abstract0_assign_linexpr_array(man, True, element, tdim, linexpr0, 1, None)
                var+=1
            dimrem = elina_dimchange_alloc(0,num_in_pixels)
            for i in range(num_in_pixels):
                dimrem.contents.dim[i] = i
            elina_abstract0_remove_dimensions(man, True, element, dimrem)
            elina_dimchange_free(dimrem)
            # handle ReLU layer 
            if(nn.layertypes[layerno]=='ReLU'):
                element = relu_box_layerwise(man,True,element,0, num_out_pixels)
            nn.ffn_counter+=1 

        else:
            print(' net type not supported')
   
    dims = elina_abstract0_dimension(man,element)
    output_size = dims.intdim + dims.realdim
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)

           
    # if epsilon is zero, try to classify else verify robustness 
    
    verified_flag = True
    predicted_label = 0
    if(LB_N0[0]==UB_N0[0]):
        for i in range(output_size):
            inf = bounds[i].contents.inf.contents.val.dbl
            flag = True
            for j in range(output_size):
                if(j!=i):
                    sup = bounds[j].contents.sup.contents.val.dbl
                    if(inf<=sup):
                        flag = False
                        break
            if(flag):
                predicted_label = i
                break    
    else:
        inf = bounds[label].contents.inf.contents.val.dbl
        for j in range(output_size):
            if(j!=label):
                sup = bounds[j].contents.sup.contents.val.dbl
                if(inf<=sup):
                    predicted_label = label
                    verified_flag = Falselse
                    break

    elina_interval_array_free(bounds,output_size)
    elina_abstract0_free(man,element)
    elina_manager_free(man)        
    return predicted_label, verified_flag

In [14]:
# def main(netname, specname, esilon, c_label = None, method = 'box'):
#     with open(netname, 'r') as netfile:
#         netstring = netfile.read()
#     with open(specname, 'r') as specfile:
#         specstring = specfile.read()
#     nn = parse_net(netstring)
#     x0_low, x0_high = parse_spec(specstring)
#     LB_N0, UB_N0 = get_perturbed_image(x0_low,0)
    
#     label, _ = analyze(nn,LB_N0,UB_N0,0) # Get label of unperturbed image, i.e. eps=0
#     start = time.time()
#     if method == 'box':
#         if(label==int(x0_low[0])):
#             LB_N0, UB_N0 = get_perturbed_image(x0_low,epsilon)
#             _, verified_flag = analyze_box(nn,LB_N0,UB_N0,label)
#             if(verified_flag):
#                 print("verified")
#             else:
#                 print("can not be verified")  
#         else:
#             print("image not correctly classified by the network. expected label ",int(x0_low[0]), " classified label: ", label)
#     if method == 'linear':
#         if(label==int(x0_low[0])):
#             LB_N0, UB_N0 = get_perturbed_image(x0_low,epsilon)
#             _, verified_flag = analyze_linear(nn,LB_N0,UB_N0,label)
#             if(verified_flag):
#                 print("verified")
#             else:
#                 print("can not be verified")  
#         else:
#             print("image not correctly classified by the network. expected label ",int(x0_low[0]), " classified label: ", label)
#     end = time.time()
#     print("analysis time: ", (end-start), " seconds")
    

In [ ]:
# def compute_rank(nn, rank_threshold, norm=1, skip_first_layer=True, skip_last_layer=True):
#     numlayer = nn.numlayer 
#     norms = np.zeros((nn.weights[1].shape[0], numlayer-2)) # TODO use list or similar if first and last layer are also used
#     for layerno in range(numlayer):
#         if skip_first_layer and layerno == 0:
#             #TODO treat first layer
#             continue
#         if skip_last_layer and layerno == numlayer-1:
#             continue
#         if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
#             weights = nn.weights[layerno]
#             biases = nn.biases[layerno]
#             norms[:, layerno-1] = np.linalg.norm(weights, ord=norm, axis=1) + biases
#         else:
#             print(' net type not supported')
    
#     rank_idxs = np.argsort(-norms, axis=None)
#     rank_idxs = rank_idxs.reshape(nn.weights[1].shape[0], int(len(rank_idxs)/ nn.weights[1].shape[0]))
    
#     for layerno in range(numlayer):
#         if skip_first_layer and layerno == 0:
#             continue
#         if skip_last_layer and layerno == numlayer-1:
#             continue
#         if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
#             nn.rank[layerno] = rank_idxs[:, layerno-1]
#             nn.use_LP[layerno] = nn.rank[layerno]  < rank_threshold
#         else:
#             print(' net type not supported')

## Define ReLU operation on abstract domain

In [25]:
def relu_linear(bounds, index):
    sup = bounds[index].contents.sup.contents.val.dbl
    inf = bounds[index].contents.inf.contents.val.dbl

    input_box = elina_box_of_abstract0(abstract)
    if(destructive):
        a = input_box
    else:
        a = elina_box_copy(man, input_box)
    print("[ReLU] Max bound of variable " + str(index) + ": " + str(a.sup[index]))
    
    a.sup[index] = max(0, sup)
    a.inf[index] = min(0, inf)
    
    return 

In [ ]:
def relu_linear_layerwise(man, destructive, abstract, start_offset, num_dim):
    end = start_offset + num_dim
    
    # get bounds of the abstract domain
    bounds = elina_abstract0_to_box(man, abstract)

    # create abstract domain to store the results
    if(destructive):
        res = abstract
    else:
        res = elina_abstract0_copy(man, abstract)
    
    # iterate over each index 
    for i in range(start_offset, end):
        res = relu_linear(bounds, i)
    
    return res

itv = elina_interval_array_alloc(num_pixels)

## Populate the interval
for i in range(num_pixels):
    elina_interval_set_double(itv[i],LB_N0[i],UB_N0[i])

## construct input abstraction
element = elina_abstract0_of_box(man, 0, num_pixels, itv)

In [ ]:
def get_bounds_using_box(man, weights, biases, previous_bounds, input_pixels):
    

In [26]:
def get_bounds_using_linear(weights, biases, previous_bounds):

# Initialize the problem variables

In [27]:
netname = '/home/riai2018/mnist_nets/mnist_relu_3_10.txt'
specname = '/home/riai2018/mnist_images/img1.txt'
epsilon = 0.001

In [28]:
with open(netname, 'r') as netfile:
    netstring = netfile.read()
with open(specname, 'r') as specfile:
    specstring = specfile.read()
nn = parse_net(netstring)
x0_low, x0_high = parse_spec(specstring)
LB_N0, UB_N0 = get_perturbed_image(x0_low,0)

In [29]:
numlayer = nn.numlayer 

for layerno in range(numlayer):
    if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
        print(nn.layertypes[layerno])

ReLU
ReLU
ReLU


# Get perturbed label (provided prediction for unperturbed is true)

In [30]:
label, _ = analyze(nn,LB_N0,UB_N0,0) # Get label of unperturbed image, i.e. eps=0

if(label == int(x0_low[0])):
    LB_N0, UB_N0 = get_perturbed_image(x0_low,epsilon)
else:
    print("image not correctly classified by the network. expected label ",int(x0_low[0]), " classified label: ", label)

# Start Verification

### 1) Define element for the input 

In [31]:
num_pixels = len(LB_N0)
numlayer = nn.numlayer 
man = elina_box_manager_alloc()
itv = elina_interval_array_alloc(num_pixels)

## Populate the interval
for i in range(num_pixels):
    elina_interval_set_double(itv[i],LB_N0[i],UB_N0[i])

## construct input abstraction
element = elina_abstract0_of_box(man, 0, num_pixels, itv)
elina_interval_array_free(itv,num_pixels)

### 2) Iterate over each layer in the network and define the neural network function

In [32]:
nn.ffn_counter = 0

for layerno in range(numlayer):
    if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
        # read the layer weights and biases
        weights = nn.weights[nn.ffn_counter]
        biases = nn.biases[nn.ffn_counter]
        np.ascontiguousarray(weights, dtype=np.double)
        np.ascontiguousarray(biases, dtype=np.double)
        
        # extract dimensions of the element
        dims = elina_abstract0_dimension(man,element)
        num_in_pixels = dims.intdim + dims.realdim
        print("[Element] Int dimensions: " + str(dims.intdim))
        print("[Element] Real dimensions: " + str(dims.realdim))
    
        # calculate number of outputs
        num_out_pixels = len(weights)
        print("[Network] Shape of weights: " + str(np.shape(weights)))
        print("[Network] Shape of biases: " + str(np.shape(biases)))
        print("[Network] Out pixels: " + str(num_out_pixels))

        ## Add the layer element to the manager 
        
        #  a) Create number of neurons in that layer and populate it
        # with the number of inputs to each neuron in the layer
        dimadd = elina_dimchange_alloc(0,num_out_pixels)    
        for i in range(num_out_pixels):
            dimadd.contents.dim[i] = num_in_pixels
        
        # b) Add dimensions to an ElinaAbstract0 pointer i.e. element
        elina_abstract0_add_dimensions(man, True, element, dimadd, False)
        elina_dimchange_free(dimadd)
        
        
        # handle affine layer
        var = num_in_pixels
        for i in range(num_out_pixels):
            # Array of ElinaDim that need to be assigned
            tdim= ElinaDim(var)
            # Create the linear expression associated to the neuron i
            linexpr0 = generate_linexpr0(weights[i], biases[i], num_in_pixels)
            # Parallel assignment of several dimensions of an ElinaAbstract0 by using an ElinaLinexpr0Array
            element = elina_abstract0_assign_linexpr_array(man, True, element, tdim, linexpr0, 1, None)
            var+=1

        # Pointer to which semantics we want to follow.
        dimrem = elina_dimchange_alloc(0,num_in_pixels)
        for i in range(num_in_pixels):
            dimrem.contents.dim[i] = i
        # Remove dimensions from an ElinaAbstract0
        elina_abstract0_remove_dimensions(man, True, element, dimrem)
        elina_dimchange_free(dimrem)
        
        # handle ReLU layer 
        element = relu_linear_layerwise(man,True,element,0, num_out_pixels)
        
        # Iterate to next layer
        nn.ffn_counter+=1 
        
        print('---------------')

    else:
        print(' net type not supported')

[Element] Int dimensions: 0
[Element] Real dimensions: 784
[Network] Shape of weights: (10, 784)
[Network] Shape of biases: (10,)
[Network] Out pixels: 10


NameError: name 'elina_box_of_abstract0' is not defined

### 3) Compute the bounds 

In [32]:
dims = elina_abstract0_dimension(man,element)
output_size = dims.intdim + dims.realdim
# get bounds for each output neuron
bounds = elina_abstract0_to_box(man,element)

## Check the verifiability of the network

In [33]:
# if epsilon is zero, try to classify else verify robustness 
verified_flag = True
predicted_label = 0
if(LB_N0[0]==UB_N0[0]):
    for i in range(output_size):
        inf = bounds[i].contents.inf.contents.val.dbl
        flag = True
        for j in range(output_size):
            if(j!=i):
                sup = bounds[j].contents.sup.contents.val.dbl
                if(inf<=sup):
                    flag = False
                    break
        if(flag):
            predicted_label = i
            break    
else:
    inf = bounds[label].contents.inf.contents.val.dbl
    for j in range(output_size):
        if(j!=label):
            sup = bounds[j].contents.sup.contents.val.dbl
            if(inf<=sup):
                predicted_label = label
                verified_flag = False
                break

elina_interval_array_free(bounds,output_size)
elina_abstract0_free(man,element)
elina_manager_free(man)   

# Check status 

In [34]:
if(verified_flag):
    print("verified")
else:
    print("can not be verified")  

verified
